In [1]:
file_path = '/Users/lfinkbeiner/Documents/GitHub/HERA/beam/'
beam_origin = 'HERA_4.9m_E-pattern_151MHz.txt'
beam_destination = 'ant1_s2'

In [2]:
%matplotlib notebook
import matplotlib.pyplot as plt
from RIMEz import beam_models
import numpy as np
import healpy as hp
from spin1_beam_model import cst_processing, jones_matrix_field
import rotations

In [3]:
processor = cst_processing.CSTDataProcessor(
    [file_path + beam_origin,
    file_path + beam_origin,
    file_path + beam_origin],
    np.array([150e6, 151e6, 152e6]),
    1, 1e-4
)

In [4]:
processor.compute_spin1_harmonics()
processor.write_model_data(file_path, beam_destination)
spline_beam_func = beam_models.model_data_to_spline_beam_func(file_path + beam_destination + '.h5', np.array([150e6, 151e6, 152e6]))

//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.pos1_Elm_samples = h5f["pos1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:19: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.neg1_Elm_samples = h5f["neg1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:20: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.freq_nodes = h5f["frequencies"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.L_model = h5f["spatial_bandlimit"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:22: H5pyDeprec

delta_nu_in is 1000000.0


In [5]:
nside = 128

theta, phi = hp.pix2ang(nside,np.arange(12 * nside * nside))
az = phi
alt = np.pi / 2 - theta
J = spline_beam_func(150e6, alt, az)
J_abs = np.abs(J)

In [7]:
def rad_interp(J_section, ra, dec):
    '''
    Abbreviation and conversion function. We get the interpolation value
    assuming @az and @alt are in degrees.
    '''
    lst_now_deg = np.degrees(rotations.get_lst())
    az, alt = rotations.eq_to_topo(ra, dec, rotations.hera_lat, lst_now_deg)
    colatitude = np.radians(90 - alt)
    longitude = np.radians(az)
    return hp.get_interp_val(J_section, colatitude, longitude)

In [10]:
# right row this represents the top ten
eq_list = []
def append_coordinate(raw_ra, raw_dec):
    angle_ra = rotations.collapse_hour(
        raw_ra[0], raw_ra[1], raw_ra[2]
    )
    angle_dec = rotations.collapse_angle(
        raw_dec[0], raw_dec[1], raw_dec[2]
    )
    eq_list.append((angle_ra, angle_dec))

In [11]:
# Name, ra, dec, (Int_Flux_Wide [mJy])

# GLEAM J052257-362727, 05 22 57.86, -36 27 27.3 (47613.247)
append_coordinate((5, 22, 57.86), (-36, 27, 27.3))

# GLEAM J131139-221640, 13 11 39.51, -22 16 40.7 (34334.873)
append_coordinate((13, 11, 39.51), (-22, 16, 40.7))

# GLEAM J044437-280948, 04 44 37.31, -28 09 48.9 (30928.867)
append_coordinate((4, 44, 37.31), (-28, 9, 48.9))

# GLEAM J183059-360229, 18 30 59.30, -36 02 29.2 (30436.060)
append_coordinate((18, 30, 59.3), (-36, 2, 29.2))

# GLEAM J210722-252556, 21 07 22.33, -25 25 56.1 (25925.882)
append_coordinate((21, 7, 22.33), (-25, 25, 56.1))

# GLEAM J090147-255516, 09 01 47.39, -25 55 16.1 (25610.060)
append_coordinate((9, 1, 47.39), (-25, 55, 16.1))

# GLEAM J203547-345403, 20 35 47.63, -34 54 03.8 (22333.044)
append_coordinate((20, 35, 47.63), (-34, 54, 3.8))

# GLEAM J190249-232949, 19 02 49.38, -23 29 49.7 (18277.397)
append_coordinate((19, 2, 49.38), (-23, 29, 49.7))

# GLEAM J002549-260211, 00 25 49.15, -26 02 11.4 (17666.073)
append_coordinate((0, 25, 49.15), (-26, 2, 11.4))

# GLEAM J235701-344532, 23 57 01.20, -34 45 32.3 (16732.759)
append_coordinate((23, 57, 1.2), (-34, 45, 32.3))

In [28]:
# string rounding abbreviation
sr = lambda v, dp : str(round(v, dp))

In [29]:
for pair in eq_list:
    print('Beam response at \nra: ' + sr(pair[0], 2) + 
          '\ndec: ' + sr(pair[1], 2))
    print('is\nxx: ' + sr(rad_interp(J_abs[:, 1, 0], pair[0], pair[1]), 5) + '\nyy: ' +
          sr(rad_interp(J_abs[:, 0, 1], pair[0], pair[1]), 5) + '\n\n')

Beam response at 
ra: 80.74
dec: -35.54
is
xx: 0.02072
yy: 0.02241


Beam response at 
ra: 197.91
dec: -21.72
is
xx: 0.0201
yy: 0.01444


Beam response at 
ra: 71.16
dec: -27.84
is
xx: 0.03453
yy: 0.03378


Beam response at 
ra: 277.75
dec: -35.96
is
xx: 0.0
yy: 0.0


Beam response at 
ra: 316.84
dec: -24.57
is
xx: 0.0
yy: 0.0


Beam response at 
ra: 135.45
dec: -24.08
is
xx: 0.2885
yy: 0.27633


Beam response at 
ra: 308.95
dec: -33.1
is
xx: 0.0
yy: 0.0


Beam response at 
ra: 285.71
dec: -22.5
is
xx: 0.0
yy: 0.0


Beam response at 
ra: 6.45
dec: -25.96
is
xx: 0.0
yy: 0.0


Beam response at 
ra: 359.25
dec: -33.24
is
xx: 0.0
yy: 0.0




In [ ]:
# what was I supposed to do, again?